손글씨 숫자 분류를 한다. 이미 학습된 매개변수를 사용하여 학습 과정은 생략하고, 추론 과정만 구현한다.<br>
이 추론 과정을 신경망의 순전파(forward propagation)이라고도 한다.<br>
<br>
기계학습과 마찬가지로 신경망도 두 단계를 거쳐 문제를 해결한다. 먼저 훈련 데이터(학습 데이터)를 사용해<br>
가중치 매개변수를 학습하고, 추론 단계에서는 앞서 학습한 매개변수를 사용하여 입력 데이터를 분류한다.

In [6]:
import sys, os
import numpy as np
sys.path.append(os.pardir) # 부모 디렉터리의 파일을 가져올 수 있도록 설정
from mnist import load_mnist
from PIL import Image

def img_show(img):
    pil_img = Image.fromarray(np.uint8(img))
    pil_img.show()

(x_train, x_test), (y_train, y_test) = load_mnist(flatten=True, normalize=False)

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

img = x_train[0]
label = y_train[0]
print(label)

print(img.shape)
img = img.reshape(28, 28)
print(img.shape)

img_show(img)

(60000, 784)
(60000,)
(10000, 784)
(10000,)
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0  84 185 159 151  60  36   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0 222 254 254 254
 254 24

##### 신경망의 추론 처리
MNIST 데이터셋을 가지고 추론을 수행하는 신경망을 구현한다.<br>
이 신경망은 입력층 뉴런을 을 784개 출력층 뉴런을 10개로 구성한다.<br>
입력층 뉴런이 784개인 이유는 이미지 크기가 28 * 28 = 784이기 때문이고, 출력층 뉴런이 10개인 이유는 이 문제가 0에서 9까지의 숫자를 구분하는 문제이기 때문이다.<br>
한편 은닉층은 총 2개로 첫 번째 은닉층에는 50개의 뉴런을 두 번째 은닉층에는 100개의 뉴런을 배치한다.<br>
여기서 50과 100은 임의로 정한 값이다.

In [7]:
import pickle

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def softmax(a):
    c = np.max(a)
    exp_a = np.exp(a - c)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a

    return y

def get_data():
    (x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, flatten=True, one_hot_label=False)
    return x_test, t_test

def init_network():
    with open("sample_weight.pkl", 'rb') as f:
        network = pickle.load(f)

    return network

def predict(network, x):
    W1, W2, W3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']

    a1 = np.dot(x, W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) + b2
    z2 = sigmoid(a2)
    a3 = np.dot(z2, W3) + b3
    y = softmax(a3)


    return y

In [9]:
x, t = get_data()
network = init_network()

accuracy_cnt = 0
for i in range(len(x)):
    y = predict(network, x[i])
    p = np.argmax(y) # 확률이 가장 높은 원소의 인덱스를 얻는다.
    if p == t[i]:
        accuracy_cnt += 1

print('Accuracy:' + str(float(accuracy_cnt) / len(x)))

Accuracy:0.9352


##### 배치 처리
입력 데이터를 하나로 묶어서 처리하는 것  
배치(batch)가 곧 묶음이라는 의미  
  
컴퓨터가 연산을 하는데 큰 이점을 준다. 입력 데이터 한 개 당 처리 시간을 대폭 줄여준다.  
쫌쫌따리로 입력데이터 한 개씩 연산하는 것보다 통으로 묶어서 한번에 하는게 효율적이다.  
대부분의 수치 계산 라이브러리가 큰 배열을 효율적으로 처리할 수 있기 때문이다.  
분할된 작은 배열 여러개를 계산하는 것 보다 큰 배열을 한꺼번에 계산하는게 속도가 빠르다.

In [10]:
x, t = get_data()
network = init_network()

batch_size = 100 # 배치 크기
accuracy_cnt = 0

for i in range(0, len(x), batch_size):
    x_batch = x[i:i+batch_size]
    y_batch = predict(network, x_batch)
    p = np.argmax(y_batch, axis=1)
    accuracy_cnt += np.sum(p == t[i:i+batch_size])

print('Accuracy:' + str(float(accuracy_cnt) / len(x)))

Accuracy:0.9352


위에 배치 안하고 짠 코드는 실행시간 2.8s  
배치 하고 짠 코드는 실행시간 0.4s

##### 정리
신경망의 순전파를 살펴봤다.  
신경망은 각 층의 뉴런들이 다음 층의 뉴런으로 신호를 전달한다는 점에서 퍼셉트론과 같다.  
하지만 다음 뉴런으로 갈 때 신호를 변화시키는 활서화 함수에 큰 차이가 있었다.  
신경망에서는 매끄럽게 변화하는 시그모이드 함수를, 퍼셉트론에서는 갑자기 변화하는 계단 함수를 활성화 함수로 사용했다.  
이 차이가 신경망 학습에 중요하다.

- 신경망에서는 활성화 함수로 시그모이드 함수와 ReLU 함수 같은 매끄럽게 변화하는 함수를 이용한다.
- 넘파이의 다차원 배열을 잘 사용하면 신경망을 효율적으로 구현할 수 있다.
- 기계학습 문제는 크게 회귀와 분류로 나눌 수 있다.
- 출력층의 활성화 함수로는 회귀에서는 주로 항등 함수를, 분류에서는 주로 소프트맥스 함수를 이용한다.
- 분류에서는 출력층의 뉴런 수를 분류하려는 클래스 수와 같게 설정한다.
- 입력 데이터를 묶은 것을 배치라 하며, 추론 처리를 이 배치 단위로 진행하면 결과를 훨씬 빠르게 얻을 수 있다.